In [1]:
%pylab inline
import networkx as nx

Populating the interactive namespace from numpy and matplotlib


##I'm taking these notes from Pissanetsky's "Sparse Matrix Technology"

###CH 7 Sparse Matrix Algebra

Ok... clearly not the best book if I have to skip so many chapters to get to the elementary material...

Let IA, JA, AN be some sparse representation of a matrix with N rows and M columns. We wish to obtain IAT, JAT, ANT, the sparse representation of the transposed matrix.

We define M lists of integers, initially empty, each with a pointer to the first free place. Another M lists of real numbers are also defined. 

Next we run over all the nonzeros of the matrix. For each nonzero I, J, we add I to the J-th list of integers and the value of the nonzero to the J-th list of real numbers, and we increment the corresponding pointer.


In [2]:
A = array([[0,0,1,0,2,3],[4,0,0,5,0,0],[0,0,6,7,0,0],[8,0,9,10,0,0],[0,11,0,0,12,13]])
A

array([[ 0,  0,  1,  0,  2,  3],
       [ 4,  0,  0,  5,  0,  0],
       [ 0,  0,  6,  7,  0,  0],
       [ 8,  0,  9, 10,  0,  0],
       [ 0, 11,  0,  0, 12, 13]])

In [15]:
class sparse(object):
    def __init__(self, rows, cols, IA, JA, AN):
        self.rows = rows
        self.cols = cols
        self.IA = IA
        self.JA = JA
        self.AN = AN
        
def make_sparse(A):
    rows, cols  = A.shape
    IA = []
    JA = []
    AN = []
    counter = 0
    for i in range(rows):
        IA.append(counter)
        for j in range(cols):
            if(A[i][j] != 0):
                counter += 1
                JA.append(j)
                AN.append(A[i][j])
    IA.append(counter)
    out = sparse(rows, cols, IA, JA, AN)
    return out
        
    
A_s = make_sparse(A)

In [24]:
def transpose(s_m):
    n = s_m.rows
    m = s_m.cols
    temp_index = [[] for i in range(m)]
    temp_values = [[] for i in range(m)]
    for row in range(n):
        for i in range(s_m.IA[row],s_m.IA[row+1]):
            temp_index[s_m.JA[i]].append(row)
            temp_values[s_m.JA[i]].append(s_m.AN[i])
    counter = 0
    IA = []
    JA = []
    AN = []
    for i in range(m):
        IA.append(counter)
        for j in range(len(temp_values[i])):
            AN.append(temp_values[i][j])
            JA.append(temp_index[i][j])
            counter += 1
    IA.append(counter)
    out = sparse(m, n, IA, JA, AN)
    return out
            
            
A_t = transpose(A_s) 
print(A_t.IA)
print(A_t.JA)
print(A_t.AN)

[0, 2, 3, 6, 9, 11, 13]
[1, 3, 4, 0, 2, 3, 1, 2, 3, 0, 4, 0, 4]
[4, 8, 11, 1, 6, 9, 5, 7, 10, 2, 12, 3, 13]


In [25]:
def make_dense(s_m):
    n = s_m.rows
    m = s_m.cols
    out = array([[0 for i in range(m)] for i in range(n)])
    for i in range(n):
        for j in range(s_m.IA[i], s_m.IA[i+1]):
            out[i][s_m.JA[j]] = s_m.AN[j]
    return out
make_dense(A_t)

array([[ 0,  4,  0,  8,  0],
       [ 0,  0,  0,  0, 11],
       [ 1,  0,  6,  9,  0],
       [ 0,  5,  7, 10,  0],
       [ 2,  0,  0,  0, 12],
       [ 3,  0,  0,  0, 13]])

In [27]:
A.T

array([[ 0,  4,  0,  8,  0],
       [ 0,  0,  0,  0, 11],
       [ 1,  0,  6,  9,  0],
       [ 0,  5,  7, 10,  0],
       [ 2,  0,  0,  0, 12],
       [ 3,  0,  0,  0, 13]])

When the above algorithm is applied twice, an ordered representation of the given matrix is obtained. Ordered representations are required for some applications, such as Gauss elimination. We note that ordering a list of n numbers usually requires n(n-1)/2 comparisons. Here, instead, ordering can be achieved with only aN+bM+cZ elementary computations, where a, b and c are small numbers and Z is the total number of nonzeros. The asymptotic complexity of the sparse algorithm is, thus, linear.

In [39]:
def add_sparse(a_s,b_s):
    n = a_s.rows
    m = a_s.cols
    IX = [None for i in range(m)]
    JC = [[] for i in range(n)]
    for row in range(n):
        for i in range(a_s.IA[row], a_s.IA[row+1]):
            IX[a_s.JA[i]] = row
            JC[row].append(a_s.JA[i])
        for i in range(b_s.IA[row], b_s.IA[row+1]):
            if(IX[b_s.JA[i]] != row):
                IX[b_s.JA[i]] = row
                JC[row].append(b_s.JA[i])
    AN = []    
    IA = []
    JA = []
    counter = 0
    for row in range(n):
        temp_x = [0 for i in range(m)] 
        IA.append(counter)
        for i in range(a_s.IA[row], a_s.IA[row+1]):
            temp_x[a_s.JA[i]] += a_s.AN[i] 
        for i in range(b_s.IA[row], b_s.IA[row+1]):
            temp_x[b_s.JA[i]] += b_s.AN[i] 
        for i in JC[row]:
            counter += 1
            AN.append(temp_x[i])
            JA.append(i)
    IA.append(counter)
    out = sparse(n, m, IA, JA, AN)
    return out
            
a = make_sparse(array([[0,0,2,0,-1,0],[4,0,3,3,7,0],[-2,0,0,0,0,-1],[0,1,0,1,0,0]]))
b = make_sparse(array([[1,0,-1,0,0,5],[0,0,0,0,-2,0],[4,6,0,2,0,0],[0,-1,1,0,0,0]]))
make_dense(add_sparse(a,b))

array([[ 1,  0,  1,  0, -1,  5],
       [ 4,  0,  3,  3,  5,  0],
       [ 2,  6,  0,  2,  0, -1],
       [ 0,  0,  1,  1,  0,  0]])

In [46]:
def multiply_matrix_vector(s_m, v):
    n = s_m.rows
    m = s_m.cols
    assert (len(v) == m), "wrong lengths!"
    out = [0 for i in range(n)]
    for row in range(n):
        for i in range(s_m.IA[row],s_m.IA[row+1]):
            out[row] += v[s_m.JA[i]]*s_m.AN[i]
    return out

multiply_matrix_vector(b,[1,2,3,4,5,6])

[28, -10, 24, 1]

In [47]:
dot(array([[1,0,-1,0,0,5],[0,0,0,0,-2,0],[4,6,0,2,0,0],[0,-1,1,0,0,0]]), array([1,2,3,4,5,6]))

array([ 28, -10,  24,   1])

In [50]:
def multiply_vector_matrix(v, s_m):
    n = s_m.rows
    m = s_m.cols
    assert (len(v) == n), "wrong lengths!"
    out = [0 for i in range(m)]
    for row in range(n):
        for i in range(s_m.IA[row],s_m.IA[row+1]):
            out[s_m.JA[i]] += v[row]*s_m.AN[i]
    return out 
multiply_vector_matrix([1,2,3,4],b)

[13, 14, 3, 6, -4, 5]

In [53]:
dot(array([1,2,3,4]), array([[1,0,-1,0,0,5],[0,0,0,0,-2,0],[4,6,0,2,0,0],[0,-1,1,0,0,0]]))

array([13, 14,  3,  6, -4,  5])

In [71]:
def multiply_matrix_matrix(a_s, b_s):
    a_n = a_s.rows
    a_m = a_s.cols
    b_n = b_s.rows
    b_m = b_s.cols
    assert (a_m == b_n), "Wrong sizes"
    
    IX = [None for i in range(b_m)]
    AN = []
    IA = []
    JA = []
    counter = 0
    for row in range(a_n):
        IA.append(counter)
        temp_x = [0 for i in range(b_m)]
        I = []
        for i in range(a_s.IA[row],a_s.IA[row+1]):
            J = a_s.JA[i]
            for j in range(b_s.IA[J],b_s.IA[J+1]):
                temp_x[b_s.JA[j]] += a_s.AN[i]*b_s.AN[j]
                if(IX[b_s.JA[j]] != row):
                    IX[b_s.JA[j]] = row
                    I.append(b_s.JA[j])
        for i in I:
            counter += 1
            JA.append(i)
            AN.append(temp_x[i])
    IA.append(counter)
    return sparse(a_n, b_m, IA, JA, AN)
    
make_dense(multiply_matrix_matrix(a,transpose(b)))

array([[ -2,   2,   0,   2],
       [  1, -14,  22,   3],
       [ -7,   0,  -8,   0],
       [  0,   0,   8,  -1]])

In [62]:
dot(make_dense(a),make_dense(b).T)

array([[ -2,   2,   0,   2],
       [  1, -14,  22,   3],
       [ -7,   0,  -8,   0],
       [  0,   0,   8,  -1]])